# Numeric SAM Demo Notebook

In [1]:
import os
import json
from pathlib import Path

from pddl_plus_parser.lisp_parsers import DomainParser, ProblemParser, TrajectoryParser

from sam_learning.learners.numeric_sam import NumericSAMLearner

os.environ["CONVEX_HULL_ERROR_PATH"] = "tests/temp_files/ch_error.txt"

In [2]:
EXAMPLES_PATH = Path("demos_data")

depot_partial_domain = DomainParser(EXAMPLES_PATH / "depot_numeric.pddl", partial_parsing=True).parse_domain()
depot_problem = ProblemParser(EXAMPLES_PATH / "pfile2.pddl", domain=depot_partial_domain).parse_problem()
depot_observation = TrajectoryParser(depot_partial_domain, depot_problem).parse_trajectory(EXAMPLES_PATH / "test_numeric_trajectory")

In [3]:
with open(EXAMPLES_PATH / "depot_fluents_map.json", "rt") as json_file:
    depot_fluents_map = json.load(json_file)


In [4]:
numeric_sam = NumericSAMLearner(depot_partial_domain, depot_fluents_map)

print(numeric_sam.partial_domain.to_pddl())

(define (domain depot)
(:requirements :typing :fluents :disjunctive-preconditions :negative-preconditions :equality)
(:types 	place locatable - object
	depot distributor - place
	truck hoist surface - locatable
	pallet crate - surface
)

(:predicates (at ?x - locatable ?y - place)
	(on ?x - crate ?y - surface)
	(in ?x - crate ?y - truck)
	(lifting ?x - hoist ?y - crate)
	(available ?x - hoist)
	(clear ?x - surface)
)

(:functions (load_limit ?t - truck)
	(current_load ?t - truck)
	(weight ?c - crate)
	(fuel-cost )
)

(:action drive
	:parameters (?x - truck ?y - place ?z - place)
	:precondition (and 
		
		
		)
	:effect (and   
		 
		))

(:action lift
	:parameters (?x - hoist ?y - crate ?z - surface ?p - place)
	:precondition (and 
		
		
		)
	:effect (and   
		 
		))

(:action drop
	:parameters (?x - hoist ?y - crate ?z - surface ?p - place)
	:precondition (and 
		
		
		)
	:effect (and   
		 
		))

(:action load
	:parameters (?x - hoist ?y - crate ?z - truck ?p - place)
	:precondition (a

In [5]:
learned_model, learning_metadata = numeric_sam.learn_action_model([depot_observation])
print(learning_metadata)

Cannot solve linear equations when too little input equations given for action - load.
Cannot solve linear equations when too little input equations given for action - unload.


{'drive': 'OK', 'lift': 'OK', 'drop': 'OK', 'load': 'not_enough_data', 'unload': 'not_enough_data'}


In [6]:
print(learned_model.to_pddl())

(define (domain depot)
(:requirements :typing :fluents :disjunctive-preconditions :negative-preconditions :equality)
(:types 	place locatable - object
	depot distributor - place
	truck hoist surface - locatable
	pallet crate - surface
)

(:predicates (at ?x - locatable ?y - place)
	(on ?x - crate ?y - surface)
	(in ?x - crate ?y - truck)
	(lifting ?x - hoist ?y - crate)
	(available ?x - hoist)
	(clear ?x - surface)
)

(:functions (load_limit ?t - truck)
	(current_load ?t - truck)
	(weight ?c - crate)
	(fuel-cost )
)

(:action drive
	:parameters (?x - truck ?y - place ?z - place)
	:precondition (and (at ?x ?y)
		
		(not (= ?y ?z))

		)
	:effect (and (at ?x ?z) (not (at ?x ?y))
		
		
		
		
(increase (fuel-cost ) 10.0)))

(:action lift
	:parameters (?x - hoist ?y - crate ?z - surface ?p - place)
	:precondition (and (clear ?y)		
(available ?x)		
(at ?z ?p)		
(on ?y ?z)		
(at ?y ?p)		
(at ?x ?p)
		
		
		)
	:effect (and (clear ?z)
		(lifting ?x ?y) (not (on ?y ?z))
		(not (available ?x))
		(